# 2017 Dataset

This dataset differs from the 2015 dataset in that the network data is stored as pcap files rather than csv files. The only problem with this dataset is that it contains no attack data. However, it will be useful as a training dataset. Pcap files are much more generalisable than csv files. The problem with the pcap files is that I need to clean the data myself! The pcap files have been converted to flows using Argus - it is these files that will be loaded here.

In [1]:
import os
import pandas as pd
import numpy as np
import pprint
from IPython.display import Markdown, display
from datetime import datetime
from datetime import timezone

##local python file holding the paths to the directories I store the log files in
import directories_to_use

### Load data into Dataframe

In [31]:
"""
Read Argus text files into one single dataframe
"""
def readDataIntoDataframe(argus_text_files_dir):
    first_time = True
    for t_file in os.listdir(argus_text_files_dir):
        if first_time:
            data2017_df = pd.read_csv(argus_text_files_dir + t_file)
            date = removeDateFromName(t_file)
            data2017_df["StartTime"] = data2017_df["StartTime"].apply(lambda x : date + x)
            data2017_df["LastTime"] = data2017_df["LastTime"].apply(lambda x : date + x)
            first_time = False
        else:
            temp_df = pd.read_csv(argus_text_files_dir + t_file)
            date = removeDateFromName(t_file)
            temp_df["StartTime"] = temp_df["StartTime"].apply(lambda x : date + x)
            temp_df["LastTime"] = temp_df["LastTime"].apply(lambda x : date + x)
            data2017_df = pd.concat([data2017_df, temp_df], ignore_index=True)
    return data2017_df

def removeDateFromName(filename):
    year = a.split("_")[-1][0:4]
    month = a.split("_")[-1][4:6]
    day = a.split("_")[-1][6:8]
    full_date = day +"-"+ month +"-"+ year + " "
    return full_date

data2017_df = readDataIntoDataframe(argus_text_files_dir)
print(data2017_df.shape)

E:\ProgramData\Anaconda2\envs\SWaT\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (20,21,107) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
E:\ProgramData\Anaconda2\envs\SWaT\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (107) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(2682823, 118)


### Clean data (i.e. remove columns that are unhelpful)

In [32]:
"""
Loops through all the columns in the dataframe and removes any that only contain nans
"""
def removeNanColumns(dataframe):
    print("Shape before: ", dataframe.shape)
    for col in dataframe:
        unique_vals = dataframe[col].unique()
        if unique_vals.shape[0] == 1:
            if np.isnan(unique_vals[0]):
                dataframe = dataframe.drop([col], axis=1)
    print("Shape after: ", dataframe.shape)
    return dataframe

"""
Remove columns that only contain one unique value.
"""
def removeSingleColumns(dataframe):
    print("Shape before: ", dataframe.shape)
    for col in dataframe:
        unique_vals = dataframe[col].unique()
        if unique_vals.shape[0] == 1:
            dataframe = dataframe.drop([col], axis=1)
    print("Shape after: ", dataframe.shape)
    return dataframe
"""
Prints the columns in a dataframe and all its unique values
"""
def printColumnsAndUniqueVals(dataframe):
    pp = pprint.PrettyPrinter(indent=4)
    for col in dataframe.columns:
        printmd("**" + col + "**: " + str(dataframe[col].unique()))
        
"""
Basically prints in markdown form, can also render HTML
"""
def printmd(string):
    display(Markdown(string))

data2017_df = removeNanColumns(data2017_df)
data2017_df = removeSingleColumns(data2017_df)
                
            

Shape before:  (2682823, 118)
Shape after:  (2682823, 70)
Shape before:  (2682823, 70)
Shape after:  (2682823, 59)


In [47]:
"""Print columns and the percentage of nans present in each column"""
data2017_mod_df.isnull().mean() * 100

StartTime      0.00000
LastTime       0.00000
Flgs           0.00000
Seq            0.00000
Dur            0.00000
RunTime        0.00000
IdleTime       0.00000
Mean           0.00000
Sum            0.00000
Min            0.00000
Max            0.00000
SrcAddr        0.00000
DstAddr        0.00000
Proto          0.00000
Sport          0.00000
Dport          0.00000
sTos           0.00000
sDSb           0.00000
sTtl           0.00000
dTtl          31.48648
sHops          0.00000
dHops         31.48648
sIpId          0.00000
Cause          0.00000
TotPkts        0.00000
SrcPkts        0.00000
DstPkts        0.00000
TotBytes       0.00000
SrcBytes       0.00000
DstBytes       0.00000
Load           0.00000
SrcLoad        0.00000
DstLoad        0.00000
Loss           0.00000
SrcLoss        0.00000
DstLoss        0.00000
pLoss          0.00000
SrcGap         0.00000
DstGap         0.00000
Rate           0.00000
SrcRate        0.00000
DstRate        0.00000
Dir            0.00000
State      

## Calculated changes made to the dataset
### Drop Columns
- **Inode** is an IP Address, so very difficult to replace. Also >90% of the values were missing - so will drop it.
- **TcpOpt** is 99% Nan so will drop that column
- **SrcTCPBase** and **DstTCPBase** are 43% missing and they refer to the base sequence number in a TCP transaction. As they are specifically related to TCP, it's not a surprise that there would be values missing since not all the traffic is TCP related. Can't replace with mean as not all traffic is TCP - if I replace with 0, it will skew data. Will have to remove the columns 
- **SrcWin** and **DstWin** seem to refer to jitter. They are quite useful but with 43% of values being Nans means that it might be misleading to include them. Will have to remove them. 
- **dTos** and **dDSb** can be removed as their only unique values are 0 and nan.
- **sIpId** and **dIpId** are not necessary. It is unique IDs that tie src, dst and port. As they contain nulls and are difficult to simply impute. They can be removed since we have all the individual elements that make up the IDs. <br>

### Drop Rows
- **SPort** and **Dport** can not be safely replaced/imputed without causing inconsistencies. Will remove rows that lack these fields - this makes up 10% of the dataset
- **sHops**, **sTtl** and **sDSb** contains only 0.5% (after removing rows from above) of nan values to might as well remove rows containing Nans for this.

### Replace with 0
- **SrcGap** and **DstGap** is 43% nan but refers to bytes missing from the stream, therefore, when Nan, we can just replace with 0. 
- **sTos** refers to Type of Service (whether traffic should take precedence etc.). Nans can be set to 0 for these fields.

### Imputed
- **dhops** will need to be imputed. Values are either 0 or 1.
- **dTtl** will also need to be imputed.


In [46]:
"""
Drop columns with too many nulls
"""
def dropChosenColumns(dataframe, column_names):
    for column in column_names: 
        dataframe = dataframe.drop([column], axis=1)
    return dataframe
        
"""
Replace Nans with 0
"""
def replaceNansWithZero(dataframe, column_names):
    for column in column_names:
        dataframe[column] = dataframe[column].fillna(0)
    return dataframe

"""
Remove rows with nans for chosen columns
"""
def removeNanRows(dataframe, column_names):
    for column in column_names:
        dataframe = dataframe[dataframe[column].notna()]
    return dataframe

data2017_mod_df = data2017_df.copy(deep=True)
data2017_mod_df = dropChosenColumns(data2017_mod_df, ['TcpOpt', 'Inode', 'SrcWin', 'DstWin', 'dTos', 'dDSb', 'SrcTCPBase', 'DstTCPBase', 'dIpId'])
data2017_mod_df = removeNanRows(data2017_mod_df, ['Sport', 'Dport', 'sIpId', 'sHops', 'sTtl', 'sDSb'])
data2017_mod_df = replaceNansWithZero(data2017_mod_df, ['SrcGap', 'DstGap', 'sTos'])

## Convert all data to numeric form so that it can be passed to ML classifiers
- **StartTime** and **LastTime** need to be converted to timestamps - though they will not be included in the classification
- **SrcAddr** and **DstAddr** need to be converted to unique values. 
- **Sport** and **Dport** needs to be cast from string and checked for hex (which would also have to be cast).
- **Flgs**, **Dir**, **Cause**, and **State** need to be converted to integers.


In [20]:
"""
Needs to know the category e.g. IPs and value to add to the dict of dicts (unique_vals)
Returns: the unique int assigned to the value
"""
def convertToNum(category, val, unique_vals):
    if category not in unique_vals:
        unique_vals[category] = {}
    if val.strip() not in unique_vals[category].keys():
        new_val = len(unique_vals[category].keys())
        unique_vals[category][val.strip()] = len(unique_vals[category].keys())
        return new_val
    else:
        return unique_vals[category][val.strip()]
    
"""
Port numbers are a special case. They're integers stored as strings.
They are either hex numbers or standard int strings - therefore, we 
need to check for hex before casting.
"""  
def convertPortToNum(val):
    isHex = '0x' in val
    if isHex:
        return int(val, base=16)
    else:
        return int(val)
    
    

"""
Takes a string in the following format:
14-06-2017 11:25:58.288831
Returns: Timestamp
"""
def createTimestamp(datetime_string):
    row_date = datetime.strptime(datetime_string, "%d-%m-%Y %H:%M:%S.%f")
    print(row_date)
    timestamp = row_date.replace(tzinfo=timezone.utc).timestamp()
    return timestamp

unique_vals = dict()

data2017_mod_df[""]
data2017_mod_df["StartTime"] = data2017_mod_df["StartTime"].apply(lambda x: createTimestamp(x))
data2017_mod_df["LastTime"] = data2017_mod_df["LastTime"].apply(lambda x: createTimestamp(x))
data2017_mod_df["SrcAddr"] = data2017_mod_df["SrcAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2017_mod_df["DstAddr"] = data2017_mod_df["DstAddr"].apply(lambda x : convertToNum("ips", x, unique_vals))
data2017_mod_df["Cause"] = data2017_mod_df["Cause"].apply(lambda x : convertToNum("cause", x, unique_vals))
data2017_mod_df["State"] = data2017_mod_df["State"].apply(lambda x : convertToNum("state", x, unique_vals))
data2017_mod_df["Flgs"] = data2017_mod_df["Flgs"].apply(lambda x : convertToNum("flgs", x, unique_vals))
data2017_mod_df["Dir"] = data2017_mod_df["Dir"].apply(lambda x : convertToNum("dir", x, unique_vals))

#Dir #need to figure out what the heck this is



0           0
1           1
2           0
3           2
4           0
           ..
2682818    11
2682819    11
2682820    11
2682821    11
2682822    11
Name: SrcAddr, Length: 2682823, dtype: int64

In [49]:
printColumnsAndUniqueVals(data2017_mod_df)

**StartTime**: ['14-06-2017 03:51:05.288831' '14-06-2017 03:51:05.289017'
 '14-06-2017 03:51:05.289227' ... '14-06-2017 08:25:07.388376'
 '14-06-2017 08:25:07.638394' '14-06-2017 08:25:07.756867']

**LastTime**: ['14-06-2017 03:51:10.284546' '14-06-2017 03:51:10.280258'
 '14-06-2017 03:51:10.288606' ... '14-06-2017 08:25:07.664191'
 '14-06-2017 08:25:07.638394' '14-06-2017 08:25:07.834927']

**Flgs**: [' e g      ' ' e        ' ' e i      ' ' e s      ' ' e r      '
 ' e d      ' ' eU       ' ' e &      ' ' e     A  ' ' e *      ']

**Seq**: [    1     2     3 ... 12059 12061 12063]

**Dur**: [4.995715 4.991427 4.999589 ... 0.80031  0.503049 0.275815]

...

**SynAck**: [0.000e+00 1.590e-04 2.140e-04 3.560e-04 1.330e-04 3.010e-04 1.610e-04
 2.190e-04 1.690e-04 1.970e-04 1.880e-04 1.660e-04 2.240e-04 2.000e-04
 1.760e-04 1.210e-04 2.380e-04 2.110e-04 3.000e-04 2.200e-04 1.290e-04
 1.470e-04 2.780e-04 1.250e-04 2.030e-04 3.230e-04 2.700e-04 7.200e-05
 2.350e-04 2.410e-04 2.270e-04 2.040e-04 2.530e-04 2.450e-04 1.920e-04
 2.100e-04 7.000e-05 9.200e-05 2.980e-04 2.650e-04 2.320e-04 2.340e-04
 2.090e-04 2.630e-04 1.100e-04 1.300e-04 2.710e-04 2.060e-04 1.350e-04
 9.800e-05 1.980e-04 1.960e-04 2.150e-04 1.870e-04 1.380e-04 1.620e-04
 1.500e-04 1.940e-04 2.470e-04 2.810e-04 2.680e-04 2.080e-04 2.860e-04
 2.160e-04 9.090e-04 2.300e-04 1.240e-04 2.180e-04 1.820e-04 1.670e-04
 2.490e-04 1.780e-04 1.600e-04 1.950e-04 2.210e-04 1.180e-04 2.280e-04
 2.760e-04 1.800e-04 2.540e-04 2.570e-04 2.940e-04 1.830e-04 1.280e-04
 2.520e-04 1.560e-04 2.580e-04 1.890e-04 2.590e-04 1.360e-04 1.400e-04
 1.340e-04 2.550e-04 2.360e-04 7.900e-05 6.700e-05 2.170e-04 2.070e-04
 1.770e-04 2.790e-04 1.490e-04 2.690e-04 1.900e-04 2.620e-04 1.750e-04
 2.770e-04 1.410e-04 1.030e-04 1.640e-04 2.600e-04 3.440e-04 1.080e-04
 1.510e-04 2.480e-04 1.730e-04 2.130e-04 2.390e-04 2.020e-04 1.480e-04
 8.300e-05 3.060e-04 1.930e-04 1.790e-04 1.990e-04 2.290e-04 8.170e-04
 3.430e-04 2.820e-04 1.720e-04 1.700e-04 5.300e-05 1.680e-04 3.020e-04
 2.840e-04 2.560e-04 1.460e-04 1.130e-04 2.640e-04 2.730e-04 2.510e-04
 2.800e-04 2.420e-04 2.610e-04 2.010e-04 1.320e-04 1.650e-04 2.440e-04
 2.500e-04 2.910e-04 1.270e-04 1.420e-04 1.370e-04 2.050e-04 2.310e-04
 2.460e-04 1.520e-04 8.080e-04 2.230e-04 2.900e-04 1.630e-04 1.740e-04
 1.110e-04 4.230e-04 2.370e-04 1.910e-04 1.390e-04 8.800e-05 1.710e-04
 7.300e-05 1.120e-04 1.090e-04 3.490e-04 8.700e-05 2.330e-04 6.600e-05
 3.190e-04 3.150e-04 3.870e-04 3.220e-04 3.100e-04 1.810e-04 2.250e-04
 2.850e-04 3.820e-04 9.330e-04 3.070e-04 2.400e-04 2.870e-04 2.660e-04
 3.050e-04 6.000e-05 2.430e-04 1.000e-04 2.720e-04 2.220e-04 3.590e-04
 7.500e-05 1.550e-04 3.160e-04 2.920e-04 2.120e-04 1.860e-04 8.490e-04
 1.840e-04 5.900e-05 2.260e-04 1.440e-04 1.580e-04 1.260e-04 2.740e-04
 1.570e-04 7.410e-04 2.880e-04 3.080e-04 1.070e-04 3.250e-04 1.850e-04
 1.060e-04 4.110e-04 1.450e-04 1.170e-04 3.450e-04 7.400e-05 8.200e-05
 9.900e-05 7.800e-05 8.730e-04 1.140e-04 1.530e-04 3.670e-04 1.310e-04
 1.230e-04 1.050e-04 3.120e-04 1.040e-04 3.170e-04 3.130e-04 1.540e-04
 8.860e-04 1.430e-04 1.190e-04 3.090e-04 2.990e-04 2.750e-04 3.460e-04
 2.830e-04 3.330e-04 2.970e-04 3.200e-04 1.200e-04 3.310e-04 2.930e-04
 8.370e-04 8.100e-05 8.000e-05 3.760e-04 2.670e-04 9.600e-05 6.500e-05
 8.140e-04 3.140e-04 8.500e-05 1.220e-04 9.300e-05 3.180e-04 8.600e-05
 2.960e-04 7.660e-04 6.200e-05 3.040e-04 3.540e-04 8.400e-05 8.890e-04
 9.000e-05 3.300e-04 7.600e-05 2.890e-04 6.760e-04 9.400e-05 3.290e-04
 9.010e-04 1.150e-04 3.380e-04 6.900e-05 6.300e-05 1.035e-03 3.570e-04
 2.950e-04 7.880e-04 3.210e-04 8.760e-04 7.100e-05 6.400e-05 3.030e-04
 9.070e-04 9.100e-05 1.020e-04 6.800e-05 8.880e-04 3.690e-04 3.110e-04
 3.360e-04 8.900e-05 7.700e-05 4.140e-04 9.640e-04 8.030e-04 3.270e-04
 1.160e-04 4.510e-04 3.960e-04 7.990e-04 3.400e-04 9.150e-04 4.020e-04
 9.500e-05 3.420e-04 3.910e-04 3.370e-04 8.260e-04 3.700e-04 7.010e-04
 4.900e-05 1.010e-04 3.390e-04 9.900e-04 9.240e-04 9.700e-05 3.640e-04
 3.320e-04 3.680e-04 3.500e-04 3.890e-04 8.120e-04 3.530e-04 9.410e-04
 3.340e-04 6.100e-05 5.600e-05 7.870e-04 8.450e-04 6.900e-04 8.670e-04
 5.500e-05 8.350e-04 8.390e-04 3.480e-04 9.390e-04 4.000e-04 8.910e-04
 4.370e-04 3.240e-04 3.470e-04 7.190e-04 3.280e-04 7.830e-04 3.260e-04
 6.950e-04 3.980e-04 9.220e-04 3.510e-04 3.810e-04 7.430e-04 7.750e-04
 3.660e-04 8.270e-04 3.790e-04 9.020e-04 3.840e-04 7.500e-04 5.800e-05
 4.310e-04 8.510e-04 8.790e-04 8.160e-04 9.350e-04 3.650e-04 8.550e-04
 2.700e-05 9.060e-04 1.029e-03 3.740e-04 4.120e-04 9.800e-04 7.450e-04
 4.040e-04 8.590e-04 3.850e-04 8.020e-04 9.690e-04 3.580e-04 8.740e-04
 5.700e-05 4.730e-04 7.090e-04 5.200e-05 3.350e-04 8.400e-04 3.830e-04
 8.770e-04 3.720e-04 3.730e-04 8.090e-04 4.260e-04 9.320e-04 9.580e-04
 7.460e-04 9.850e-04 4.060e-04 7.920e-04 3.410e-04 9.080e-04 8.340e-04
 4.520e-04 9.780e-04 4.160e-04 8.470e-04 8.930e-04 8.630e-04 3.630e-04
 3.550e-04 3.930e-04 8.610e-04 8.820e-04 8.300e-04 8.850e-04 7.140e-04
 9.300e-04 7.560e-04 8.220e-04 9.210e-04 8.640e-04 3.620e-04 7.840e-04
 5.320e-04 4.570e-04 8.600e-04 3.842e-03 3.610e-04 3.880e-04 3.920e-04
 1.021e-03 7.550e-04 9.430e-04 6.840e-04 7.910e-04 8.380e-04 7.330e-04
 8.690e-04 9.190e-04 3.770e-04 8.210e-04 7.700e-04 9.170e-04 7.900e-04
 3.520e-04 8.750e-04 8.500e-04 7.290e-04 8.130e-04 7.540e-04 7.230e-04
 7.610e-04 9.760e-04 2.406e-03 9.050e-04 8.920e-04 5.400e-05 9.480e-04
 4.190e-04 3.600e-04 8.900e-04 9.120e-04 6.700e-04 4.170e-04 7.790e-04
 9.250e-04 9.660e-04 6.970e-04 9.400e-04 8.810e-04 8.870e-04 7.850e-04
 8.460e-04 8.040e-04 1.008e-03 4.780e-04 3.860e-04 7.770e-04 7.620e-04
 3.750e-04 6.890e-04 9.560e-04 9.180e-04 9.420e-04 7.950e-04 3.800e-04
 8.520e-04 1.023e-03 8.190e-04 5.820e-04 9.970e-04 9.280e-04 5.170e-04
 4.640e-04 9.680e-04 7.680e-04 9.870e-04 4.050e-04 6.270e-04 4.540e-04
 9.770e-04 8.050e-04 4.430e-04 8.410e-04 8.960e-04 7.780e-04 9.360e-04
 7.070e-04 8.180e-04 7.650e-04 1.006e-03 4.090e-04 8.800e-04 4.360e-04
 9.100e-04 6.930e-04 9.570e-04 9.440e-04 8.950e-04 9.750e-04 4.400e-05
 8.650e-04 7.730e-04 4.390e-04 5.100e-05 9.820e-04 6.650e-04 2.609e-03
 1.002e-03 4.820e-04 1.940e-03 3.780e-04 9.610e-04 6.810e-04 9.630e-04
 7.100e-04 7.720e-04 9.700e-04 4.200e-05 8.680e-04 8.150e-04 9.140e-04
 8.440e-04 4.130e-04 9.670e-04 3.970e-04 3.990e-04 8.480e-04 4.700e-05
 8.110e-04 4.340e-04 9.710e-04 7.760e-04 1.027e-03 7.160e-04 8.250e-04
 8.310e-04 7.890e-04 8.530e-04 9.370e-04 9.310e-04 4.470e-04 7.970e-04
 8.940e-04 3.710e-04 3.940e-04 1.040e-03 9.510e-04 9.200e-04 4.620e-04
 7.350e-04 8.010e-04 5.000e-05 7.630e-04 8.060e-04 9.950e-04 3.600e-05
 4.800e-05 7.420e-04 7.520e-04 7.670e-04 4.590e-04 6.850e-04 8.360e-04
 8.970e-04 8.070e-04 9.990e-04 9.740e-04 7.170e-04 4.785e-03 4.217e-03
 4.030e-04 9.520e-04 9.130e-04 5.380e-04 2.600e-05 4.330e-04 2.400e-05
 3.400e-05 4.500e-05 8.720e-04 5.430e-04 5.640e-04 2.924e-03 9.380e-04
 7.300e-04 4.320e-04 9.590e-04 8.660e-04 3.950e-04 8.580e-04 6.600e-04
 7.120e-04 7.940e-04 7.820e-04 1.065e-03 8.320e-04 6.490e-04 4.400e-04
 9.530e-04 9.550e-04 1.036e-03 9.270e-04 9.260e-04 7.930e-04 9.040e-04
 5.450e-04 4.960e-04 4.100e-05 4.850e-04 4.670e-04 3.900e-05 9.160e-04
 8.430e-04 4.750e-04 7.640e-04 4.410e-04 7.980e-04 2.194e-03 4.010e-04
 8.570e-04 7.740e-04 8.230e-04 4.600e-05 1.126e-03 8.700e-04 5.010e-04
 8.780e-04 4.940e-04 2.900e-05 8.420e-04 9.340e-04 3.800e-05 9.030e-04
 7.810e-04 8.200e-04 6.980e-04 9.230e-04 1.028e-03 7.110e-04 7.580e-04
 9.860e-04 2.874e-03 6.220e-04 4.240e-04 1.044e-03 9.650e-04 3.900e-04
 3.358e-03 7.320e-04 9.890e-04 4.070e-04 1.057e-03 4.000e-06 4.660e-04
 4.650e-04 9.460e-04 1.010e-03 9.540e-04 8.240e-04 7.060e-04 5.660e-04
 4.270e-04 8.620e-04 9.600e-04 5.210e-04 4.220e-04 4.180e-04 4.690e-04
 7.050e-04 1.007e-03 5.990e-04 4.460e-04 8.560e-04 5.550e-04 6.940e-04
 2.100e-05 8.980e-04 4.250e-04 8.330e-04 7.510e-04 1.030e-03 7.590e-04
 8.290e-04 5.060e-04 4.350e-04 9.110e-04 8.540e-04 9.490e-04 5.360e-04
 7.280e-04 2.830e-03 6.820e-04 1.900e-05 9.470e-04 1.146e-03 7.240e-04
 1.162e-03 9.620e-04 6.830e-04 8.840e-04 1.429e-03 6.250e-04 6.320e-04
 4.200e-04 9.500e-04 4.080e-04 9.720e-04 4.790e-04 9.910e-04 1.046e-03
 5.020e-04 1.014e-03 4.150e-04 4.100e-04 7.690e-04 4.630e-04 2.800e-05
 4.210e-04 7.210e-04 3.100e-05 1.253e-03 1.500e-05 5.850e-04 4.420e-04
 4.530e-04 1.340e-03 4.703e-03 4.811e-03 2.727e-03 2.000e-06 4.085e-03
 4.290e-04 5.047e-03 3.205e-03 4.300e-05 3.000e-05 5.130e-04 4.580e-04
 3.700e-05 5.080e-04 4.000e-05 5.030e-04 6.070e-04 5.530e-04 5.100e-04
 4.560e-04 1.000e-06 3.200e-05 5.970e-04 6.920e-04 3.500e-05 5.110e-04
 1.403e-03 4.380e-04 4.300e-04 5.890e-04 6.540e-04 4.610e-04 5.690e-04
 7.030e-04 4.860e-04 4.280e-04 3.634e-03 7.530e-04]

**AckDat**: [0.       0.000302 0.000405 ... 0.001824 0.001104 0.000912]

**Offset**: [    128     348     568 ... 3422928 3423140 3423352]

**sMeanPktSz**: [ 88.013885  88.030899  88.033043 ...  88.25      88.26136  886.095215]

**dMeanPktSz**: [101.987373  88.006866  98.693962 ... 101.861115  87.86364  286.333344]

In [52]:
s = '56790'
int(s)

56790